In [15]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import nbimporter
from fetch_data_from_yfinance import data_puller
import util_data_cleaner
import warnings
warnings.filterwarnings("ignore")

In [2]:
def detect_frequency(date_df):
    date_diffs = date_df.diff().dropna()
    median_diff = date_diffs.median()
    
    daily_threshold = pd.Timedelta(days=1.5)
    weekly_threshold = pd.Timedelta(weeks=1.5)
    
    # Determine frequency based on the median difference
    if median_diff <= daily_threshold:
        return('1d')
    elif median_diff <= weekly_threshold:
        return('1wk')
    else:
        return('1mo')

In [36]:
def adding_incremental_data():
    
    existing_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\stock_data_raw.csv.gz', compression='gzip')
    existing_data = util_data_cleaner.raw_data_cleaner(existing_data)
    stock_universe = existing_data['Stock Name'].unique().tolist()
    dates = pd.to_datetime(existing_data.loc[existing_data['Stock Name']==stock_universe[0],'Date'])
    
    start_date = dates.max() + timedelta(days=1) 
    end_date = date.today()
    interval = detect_frequency(dates)
    inter_df = []
    
    if(start_date != end_date):
        for i in stock_universe:
            df = data_puller(i, start_date, end_date, interval)
            df.reset_index(inplace=True)
            inter_df.append(df)
        final_data = pd.concat(inter_df)
        final_data = pd.concat([existing_data, final_data])
        final_data.reset_index(inplace=True)
        final_data.to_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\stock_data_raw.csv.gz', index=False, compression='gzip')

In [37]:
adding_incremental_data()


1 Failed download:
['SICALLOG.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-08-20 00:00:00 -> 2024-08-23)')
